# Exploring and Clustering neighborhoods in Toronto

In this notebook we explore neighborhoods belonging to the Toronto Downtown borough.

In [3]:
import numpy as np
import pandas as pd

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [4]:
# Import neighborhoods as previsouly exported
neighborhoods = pd.read_csv('postCodes_Coords.csv')
neighborhoods

Unnamed: 0  Unnamed: 0.1 PostalCode           Borough  \
0             0             0        M3A        North York   
1             1             1        M4A        North York   
2             2             2        M5A  Downtown Toronto   
3             3             3        M6A        North York   
4             4             4        M7A  Downtown Toronto   
..          ...           ...        ...               ...   
98           98            98        M8X         Etobicoke   
99           99            99        M4Y  Downtown Toronto   
100         100           100        M7Y      East Toronto   
101         101           101        M8Y         Etobicoke   
102         102           102        M8Z         Etobicoke   

                                          Neighborhood   Latitude  Longitude  
0                                            Parkwoods  43.753259 -79.329656  
1                                     Victoria Village  43.725882 -79.315572  
2                            Regent Park, Harbourfront  43.654260 -79.360636  
3                     Lawrence Manor, Lawrence Heights  43.718518 -79.464763  
4          Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
..                                                 ...        ...        ...  
98       The Kingsway, Montgomery Road, Old Mill North  43.653654 -79.506944  
99                                Church and Wellesley  43.665860 -79.383160  
100  Business reply mail Processing Centre, South C...  43.662744 -79.321558  
101  Old Mill South, King's Mill Park, Sunnylea, Hu...  43.636258 -79.498509  
102  Mimico NW, The Queensway West, South of Bloor,...  43.628841 -79.520999  

[103 rows x 7 columns]

In [5]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [6]:
address = 'TORONTO, ONTARIO'

geolocator = Nominatim(user_agent="to_explorer_mdc")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add neighborhood markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='black',
        fill_opacity=0.4,
        parse_html=False).add_to(map_toronto)
    
map_toronto

Let us have a more detailed look at toronto downtown

In [10]:
# Keeping neighborhoods having Toronto Downtown as a borough
downtown_data = neighborhoods[neighborhoods['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
downtown_data.head()

Unnamed: 0  Unnamed: 0.1 PostalCode           Borough  \
0           2             2        M5A  Downtown Toronto   
1           4             4        M7A  Downtown Toronto   
2           9             9        M5B  Downtown Toronto   
3          15            15        M5C  Downtown Toronto   
4          20            20        M5E  Downtown Toronto   

                                  Neighborhood   Latitude  Longitude  
0                    Regent Park, Harbourfront  43.654260 -79.360636  
1  Queen's Park, Ontario Provincial Government  43.662301 -79.389494  
2                     Garden District, Ryerson  43.657162 -79.378937  
3                               St. James Town  43.651494 -79.375418  
4                                  Berczy Park  43.644771 -79.373306

In [11]:
address = 'TORONTO, ONTARIO'

geolocator = Nominatim(user_agent="dt_toronto_explorer_mdc")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of downtown Toronto are 43.6534817, -79.3839347.


In [29]:
# create map of Manhattan using latitude and longitude values
map_dtToronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label in zip(downtown_data['Latitude'], downtown_data['Longitude'], downtown_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='black',
        fill_opacity=0.4,
        parse_html=False).add_to(map_dtToronto)  
    
map_dtToronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version

In [29]:
# TODO Fill in your own credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: FQ1E0INPROWL4GNUPJAS2KRK25VPZCOHODUJIHQBQQL40RGQ
CLIENT_SECRET:I0YEZNABB3FHMXL2OI4GC1BUD5FEWP5ERDBYVEPGNDSV1NTK


In [30]:
# Having a look at the first neighborhood of Toronto downtown
downtown_data.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [31]:
neighborhood_latitude = downtown_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = downtown_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = downtown_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [32]:
# Building up a query to the foursquares API

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [33]:
results = requests.get(url).json()

In [34]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name           categories        lat        lng
0        Roselle Desserts               Bakery  43.653447 -79.362017
1           Tandem Coffee          Coffee Shop  43.653559 -79.361809
2  Cooper Koo Family YMCA  Distribution Center  43.653249 -79.358008
3      Morning Glory Cafe       Breakfast Spot  43.653947 -79.361149
4     Body Blitz Spa East                  Spa  43.654735 -79.359874

In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


# Explore neighborhoods in downtown Toronto

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    '''    
    Return the nearby (within radius) venues
            Parameters:
                    names (list of str): A list of neighborhood names
                    latitudes (list of floats): A list of latitudes corresponding to the aforementioned names
                    longitudes (list of floats): A list of latitudes corresponding to the aforementioned names
                    radius (int): radius in meters

            Returns:
                    nearby_venues (pandas dataframe): A pandas dataframe of nearby venues
    '''  
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

In [40]:
dtToronto_venues = getNearbyVenues(names=downtown_data['Neighborhood'],
                                   latitudes=downtown_data['Latitude'],
                                   longitudes=downtown_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


In [41]:
print(dtToronto_venues.shape)
dtToronto_venues.head()

(1224, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  Regent Park, Harbourfront               43.65426              -79.360636   
1  Regent Park, Harbourfront               43.65426              -79.360636   
2  Regent Park, Harbourfront               43.65426              -79.360636   
3  Regent Park, Harbourfront               43.65426              -79.360636   
4  Regent Park, Harbourfront               43.65426              -79.360636   

                    Venue  Venue Latitude  Venue Longitude  \
0        Roselle Desserts       43.653447       -79.362017   
1           Tandem Coffee       43.653559       -79.361809   
2  Cooper Koo Family YMCA       43.653249       -79.358008   
3      Morning Glory Cafe       43.653947       -79.361149   
4     Body Blitz Spa East       43.654735       -79.359874   

        Venue Category  
0               Bakery  
1          Coffee Shop  
2  Distribution Center  
3       Breakfast Spot  
4                  Spa

In [43]:
# Counting the number of venues per neighborhood of downtown Toronto
dtToronto_venues.groupby('Neighborhood').count()

Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            58   
CN Tower, King and Spadina, Railway Lands, Harb...                     17   
Central Bay Street                                                     64   
Christie                                                               16   
Church and Wellesley                                                   77   
Commerce Court, Victoria Hotel                                        100   
First Canadian Place, Underground city                                100   
Garden District, Ryerson                                              100   
Harbourfront East, Union Station, Toronto Islands                     100   
Kensington Market, Chinatown, Grange Park                              61   
Queen's Park, Ontario Provincial Government                            33   
Regent Park, Harbourfront                                              45   
Richmond, Adelaide, King                                               96   
Rosedale                                                                4   
St. James Town                                                         80   
St. James Town, Cabbagetown                                            43   
Stn A PO Boxes                                                         98   
Toronto Dominion Centre, Design Exchange                              100   
University of Toronto, Harbord                                         32   

                                                    Neighborhood Longitude  \
Neighborhood                                                                 
Berczy Park                                                             58   
CN Tower, King and Spadina, Railway Lands, Harb...                      17   
Central Bay Street                                                      64   
Christie                                                                16   
Church and Wellesley                                                    77   
Commerce Court, Victoria Hotel                                         100   
First Canadian Place, Underground city                                 100   
Garden District, Ryerson                                               100   
Harbourfront East, Union Station, Toronto Islands                      100   
Kensington Market, Chinatown, Grange Park                               61   
Queen's Park, Ontario Provincial Government                             33   
Regent Park, Harbourfront                                               45   
Richmond, Adelaide, King                                                96   
Rosedale                                                                 4   
St. James Town                                                          80   
St. James Town, Cabbagetown                                             43   
Stn A PO Boxes                                                          98   
Toronto Dominion Centre, Design Exchange                               100   
University of Toronto, Harbord                                          32   

                                                    Venue  Venue Latitude  \
Neighborhood                                                                
Berczy Park                                            58              58   
CN Tower, King and Spadina, Railway Lands, Harb...     17              17   
Central Bay Street                                     64              64   
Christie                                               16              16   
Church and Wellesley                                   77              77   
Commerce Court, Victoria Hotel                        100             100   
First Canadian Place, Underground city                100             100   
Garden District, Ryerson                              100             100   
Harbourfront East, Union 

In [44]:
print('There are {} uniques categories.'.format(len(dtToronto_venues['Venue Category'].unique())))

There are 206 uniques categories.


# 3 Analyze Each Neighborhood of downtown Toronto

In [59]:
# one hot encoding of the categories in which the amenities are falling.
dtToronto_onehot = pd.get_dummies(dtToronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dtToronto_onehot['Neighborhood'] = dtToronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dtToronto_onehot.columns[-1]] + list(dtToronto_onehot.columns[:-1])
dtToronto_onehot = dtToronto_onehot[fixed_columns]

dtToronto_onehot.head()


Yoga Studio  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0            0        0                   0             0               0   
1            0        0                   0             0               0   
2            0        0                   0             0               0   
3            0        0                   0             0               0   
4            0        0                   0             0               0   

   Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
0                0                 0                    0             0   
1                0                 0                    0             0   
2                0                 0                    0             0   
3                0                 0                    0             0   
4                0                 0                    0             0   

   Aquarium  ...  Tea Room  Thai Restaurant  Theater  Theme Restaurant  Trail  \
0         0  ...         0                0        0                 0      0   
1         0  ...         0                0        0                 0      0   
2         0  ...         0                0        0                 0      0   
3         0  ...         0                0        0                 0      0   
4         0  ...         0                0        0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  
0                      0         0  
1                      0         0  
2                      0         0  
3                      0         0  
4                      0         0  

[5 rows x 206 columns]

In [60]:
# This should be a nVenues * nCategories dataframe
dtToronto_onehot.shape

(1224, 206)

In [61]:
dtToronto_grouped = dtToronto_onehot.groupby('Neighborhood').mean().reset_index()
dtToronto_grouped

Neighborhood  Yoga Studio   Airport  \
0                                         Berczy Park     0.000000  0.000000   
1   CN Tower, King and Spadina, Railway Lands, Har...     0.000000  0.058824   
2                                  Central Bay Street     0.015625  0.000000   
3                                            Christie     0.000000  0.000000   
4                                Church and Wellesley     0.025974  0.000000   
5                      Commerce Court, Victoria Hotel     0.000000  0.000000   
6              First Canadian Place, Underground city     0.000000  0.000000   
7                            Garden District, Ryerson     0.000000  0.000000   
8   Harbourfront East, Union Station, Toronto Islands     0.000000  0.000000   
9           Kensington Market, Chinatown, Grange Park     0.000000  0.000000   
10        Queen's Park, Ontario Provincial Government     0.030303  0.000000   
11                          Regent Park, Harbourfront     0.022222  0.000000   
12                           Richmond, Adelaide, King     0.000000  0.000000   
13                                           Rosedale     0.000000  0.000000   
14                                     St. James Town     0.000000  0.000000   
15                        St. James Town, Cabbagetown     0.000000  0.000000   
16                                     Stn A PO Boxes     0.010204  0.000000   
17           Toronto Dominion Centre, Design Exchange     0.000000  0.000000   
18                     University of Toronto, Harbord     0.031250  0.000000   

    Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0             0.000000      0.000000        0.000000         0.000000   
1             0.058824      0.058824        0.117647         0.117647   
2             0.000000      0.000000        0.000000         0.000000   
3             0.000000      0.000000        0.000000         0.000000   
4             0.000000      0.000000        0.000000         0.000000   
5             0.000000      0.000000        0.000000         0.000000   
6             0.000000      0.000000        0.000000         0.000000   
7             0.000000      0.000000        0.000000         0.000000   
8             0.000000      0.000000        0.000000         0.000000   
9             0.000000      0.000000        0.000000         0.000000   
10            0.000000      0.000000        0.000000         0.000000   
11            0.000000      0.000000        0.000000         0.000000   
12            0.000000      0.000000        0.000000         0.000000   
13            0.000000      0.000000        0.000000         0.000000   
14            0.000000      0.000000        0.000000         0.000000   
15            0.000000      0.000000        0.000000         0.000000   
16            0.000000      0.000000        0.000000         0.000000   
17            0.000000      0.000000        0.000000         0.000000   
18            0.000000      0.000000        0.000000         0.000000   

    Airport Terminal  American Restaurant  Antique Shop  ...  Tea Room  \
0           0.000000             0.000000      0.000000  ...      0.00   
1           0.117647             0.000000      0.000000  ...      0.00   
2           0.000000             0.000000      0.000000  ...      0.00   
3           0.000000             0.000000      0.000000  ...      0.00   
4           0.000000             0.012987      0.000000  ...      0.00   
5           0.000000             0.040000      0.000000  ...      0.01   
6           0.000000             0.030000      0.000000  ...      0.01   
7           0.000000             0.000000      0.000000  ...      0.01   
8           0.000000             0.000000      0.000000  ...      0.01   
9           0.000000             0.000000      0.000000  ...      0.00   
10          0.000000             0.000000      0.000000  ...      0.00   
11          0.000000             0.000000      0.022222  ...      0.00   
12          0.000

In [62]:
dtToronto_grouped.shape

(19, 206)

In [63]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in dtToronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dtToronto_grouped[dtToronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.03
3      Farmers Market  0.03
4          Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3       Coffee Shop  0.06
4   Harbor / Marina  0.06


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.17
1                Café  0.06
2      Sandwich Place  0.06
3  Italian Restaurant  0.06
4         Salad Place  0.03


----Christie----
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3  Athletics & Sports  0.06
4          Restaurant  0.06


----Church and Wellesley----
                  venue  freq
0           Coffee Shop  0.09
1   Japanese Restaurant  0.06
2      Sushi Restaurant  0.06
3   

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [76]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dtToronto_grouped['Neighborhood']

for ind in np.arange(dtToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dtToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                                        Berczy Park           Coffee Shop   
1  CN Tower, King and Spadina, Railway Lands, Har...        Airport Lounge   
2                                 Central Bay Street           Coffee Shop   
3                                           Christie         Grocery Store   
4                               Church and Wellesley           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0          Cocktail Bar           Cheese Shop        Farmers Market   
1       Airport Service      Airport Terminal              Boutique   
2    Italian Restaurant                  Café        Sandwich Place   
3                  Café                  Park            Baby Store   
4   Japanese Restaurant      Sushi Restaurant            Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant    Seafood Restaurant                Bakery   
1         Boat or Ferry                 Plane   Rental Car Location   
2           Salad Place       Bubble Tea Shop          Burger Joint   
3             Nightclub           Coffee Shop    Italian Restaurant   
4               Gay Bar  Fast Food Restaurant           Yoga Studio   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0              Beer Bar       Thai Restaurant                  Beach  
1      Sculpture Garden       Harbor / Marina            Coffee Shop  
2    Miscellaneous Shop   Japanese Restaurant              Juice Bar  
3            Restaurant    Athletics & Sports            Candy Store  
4                 Hotel                  Café            Men's Store

# Clustering Neighborhoods

In [77]:
# set number of clusters
kclusters = 6

dtToronto_grouped_clustering = dtToronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dtToronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 0, 4, 1, 1, 1, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [78]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtToronto_merged = downtown_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
dtToronto_merged = dtToronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

dtToronto_merged # check the last columns!

Unnamed: 0  Unnamed: 0.1 PostalCode           Borough  \
0            2             2        M5A  Downtown Toronto   
1            4             4        M7A  Downtown Toronto   
2            9             9        M5B  Downtown Toronto   
3           15            15        M5C  Downtown Toronto   
4           20            20        M5E  Downtown Toronto   
5           24            24        M5G  Downtown Toronto   
6           25            25        M6G  Downtown Toronto   
7           30            30        M5H  Downtown Toronto   
8           36            36        M5J  Downtown Toronto   
9           42            42        M5K  Downtown Toronto   
10          48            48        M5L  Downtown Toronto   
11          80            80        M5S  Downtown Toronto   
12          84            84        M5T  Downtown Toronto   
13          87            87        M5V  Downtown Toronto   
14          91            91        M4W  Downtown Toronto   
15          92            92        M5W  Downtown Toronto   
16          96            96        M4X  Downtown Toronto   
17          97            97        M5X  Downtown Toronto   
18          99            99        M4Y  Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  \
0                           Regent Park, Harbourfront  43.654260 -79.360636   
1         Queen's Park, Ontario Provincial Government  43.662301 -79.389494   
2                            Garden District, Ryerson  43.657162 -79.378937   
3                                      St. James Town  43.651494 -79.375418   
4                                         Berczy Park  43.644771 -79.373306   
5                                  Central Bay Street  43.657952 -79.387383   
6                                            Christie  43.669542 -79.422564   
7                            Richmond, Adelaide, King  43.650571 -79.384568   
8   Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752   
9            Toronto Dominion Centre, Design Exchange  43.647177 -79.381576   
10                     Commerce Court, Victoria Hotel  43.648198 -79.379817   
11                     University of Toronto, Harbord  43.662696 -79.400049   
12          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049   
13  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420   
14                                           Rosedale  43.679563 -79.377529   
15                                     Stn A PO Boxes  43.646435 -79.374846   
16                        St. James Town, Cabbagetown  43.667967 -79.367675   
17             First Canadian Place, Underground city  43.648429 -79.382280   
18                               Church and Wellesley  43.665860 -79.383160   

    Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0                1           Coffee Shop                   Pub   
1                0           Coffee Shop      Sushi Restaurant   
2                1           Coffee Shop        Clothing Store   
3                1           Coffee Shop                  Café   
4                1           Coffee Shop          Cocktail Bar   
5                0           Coffee Shop    Italian Restaurant   
6                4         Grocery Store                  Café   
7                1           Coffee Shop                  Café   
8                1           Coffee Shop              Aquarium   
9                1           Coffee Shop                 Hotel   
10               1           Coffee Shop            Restaurant   
11               5                  Café   Japanese Restaurant   
12               1                  Café           Coffee Shop   
13               3        Airport Lounge       Airport Service   
14               2                  Park                 Trail   
15               1           Coffee Shop    Seafood Restaurant   
16               1           Coffee Shop           Pizza Place   
17            

In [79]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtToronto_merged['Latitude'], dtToronto_merged['Longitude'], dtToronto_merged['Neighborhood'], dtToronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [80]:
# Cluster 1
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 0, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

Unnamed: 0.1   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
1             4  43.662301 -79.389494               0           Coffee Shop   
5            24  43.657952 -79.387383               0           Coffee Shop   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1      Sushi Restaurant           Yoga Studio        Discount Store   
5    Italian Restaurant                  Café        Sandwich Place   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
1                   Bar         Smoothie Shop    Italian Restaurant   
5           Salad Place       Bubble Tea Shop          Burger Joint   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
1              Beer Bar        Sandwich Place    Distribution Center  
5    Miscellaneous Shop   Japanese Restaurant              Juice Bar

In [81]:
# Cluster 2
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 1, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

Unnamed: 0.1   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0              2  43.654260 -79.360636               1           Coffee Shop   
2              9  43.657162 -79.378937               1           Coffee Shop   
3             15  43.651494 -79.375418               1           Coffee Shop   
4             20  43.644771 -79.373306               1           Coffee Shop   
7             30  43.650571 -79.384568               1           Coffee Shop   
8             36  43.640816 -79.381752               1           Coffee Shop   
9             42  43.647177 -79.381576               1           Coffee Shop   
10            48  43.648198 -79.379817               1           Coffee Shop   
12            84  43.653206 -79.400049               1                  Café   
15            92  43.646435 -79.374846               1           Coffee Shop   
16            96  43.667967 -79.367675               1           Coffee Shop   
17            97  43.648429 -79.382280               1           Coffee Shop   
18            99  43.665860 -79.383160               1           Coffee Shop   

   2nd Most Common Venue          3rd Most Common Venue  \
0                    Pub                           Park   
2         Clothing Store            Japanese Restaurant   
3                   Café                      Gastropub   
4           Cocktail Bar                    Cheese Shop   
7                   Café                     Restaurant   
8               Aquarium                           Café   
9                  Hotel                           Café   
10            Restaurant                          Hotel   
12           Coffee Shop  Vegetarian / Vegan Restaurant   
15    Seafood Restaurant                       Beer Bar   
16           Pizza Place                           Café   
17                  Café                          Hotel   
18   Japanese Restaurant               Sushi Restaurant   

    4th Most Common Venue      5th Most Common Venue 6th Most Common Venue  \
0                  Bakery                    Theater        Breakfast Spot   
2         Bubble Tea Shop  Middle Eastern Restaurant        Cosmetics Shop   
3            Cocktail Bar        American Restaurant      Department Store   
4          Farmers Market                 Restaurant    Seafood Restaurant   
7          Clothing Store                     Bakery       Thai Restaurant   
8                   Hotel         Italian Restaurant        Scenic Lookout   
9      Seafood Restaurant        American Restaurant           Salad Place   
10                   Café        American Restaurant                   Gym   
12  Vietnamese Restaurant         Mexican Restaurant   Arts & Crafts Store   
15           Cocktail Bar        Japanese Restaurant                  Café   
16     Chinese Restaurant                     Bakery                   Pub   
17    Japanese Restaurant                        Gym            Restaurant   
18             Restaurant                    Gay Bar  Fast Food Restaurant   

   7th Most Common Venue 8th Most Common Venue  9th Most Common Venue  \
0                   Café   Distribution Center  Performing Arts Venue   
2                   Café                 Hotel            Pizza Place   
3             Restaurant                  Park               Beer Bar   
4                 Bakery              Beer Bar        Thai Restaurant   
7                    Gym         Deli / Bodega                  Hotel   
8                Brewery   Fried Chicken Joint             Restaurant   
9     Italian Restaurant            Restaurant    Japanese Restaurant   
10    Italian Restaurant   Japanese Restaurant     Seafood Restaurant   
12         Grocery Store                   Bar         Farmers Market   
15    Italian Restaurant            Restaurant                  Hotel   
16    Italian Restaurant            Restaurant          Deli / Bodega   
17           Salad Place         Deli / Bodega             Steakhouse   
18           Y

In [83]:
# Cluster 3
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 2, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

Unnamed: 0.1   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
14            91  43.679563 -79.377529               2                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14                 Trail            Playground          Dance Studio   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
14   Dumpling Restaurant            Donut Shop      Doner Restaurant   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
14               Dog Run   Distribution Center         Discount Store

In [84]:
# Cluster 4
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 3, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

Unnamed: 0.1   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
13            87  43.628947  -79.39442               3        Airport Lounge   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
13       Airport Service      Airport Terminal              Boutique   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
13         Boat or Ferry                 Plane   Rental Car Location   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
13      Sculpture Garden       Harbor / Marina            Coffee Shop

In [85]:
# Cluster 5
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 4, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

Unnamed: 0.1   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
6            25  43.669542 -79.422564               4         Grocery Store   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
6                  Café                  Park            Baby Store   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
6             Nightclub           Coffee Shop    Italian Restaurant   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
6            Restaurant    Athletics & Sports            Candy Store

In [86]:
# Cluster 6
dtToronto_merged.loc[dtToronto_merged['Cluster Labels'] == 5, dtToronto_merged.columns[[1] + list(range(5, dtToronto_merged.shape[1]))]]

Unnamed: 0.1   Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
11            80  43.662696 -79.400049               5                  Café   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
11   Japanese Restaurant             Bookstore                   Bar   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
11                Bakery    Italian Restaurant              Beer Bar   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
11            Beer Store           College Gym         Sandwich Place